## City to Facebook ID mapping - Marketplace
This Jupiter notebook provides a mapping of postal code and country with Facebook marketplace cities ID
It reads input cities information as CSV/JSON/XML data.
It crawls Facebook graphql API to extract cities information.
It creates a python dictionary that can be easily embedded in a program.

1. Fill input form information

In [53]:
#@title Input Form

FileType = "CSV" #@param ["CSV", "JSON", "XML"]
FileName = 'data.csv' #@param {type:"string"}

2. Import postal code information file (csv/json/xml)

In [54]:
import pandas as pd
if FileType == "CSV":
    df = pd.read_csv(FileName, sep=";")

if FileType == "JSON":
    df = pd.read_json(FileName)
    
if FileType == "XML":
    df = pd.read_xml(FileName)
    
df.head()

,Codice Regione,Codice Citt� Metropolitana,Codice Provincia (1),Progressivo del Comune (2),Codice Comune formato alfanumerico,place name,Denominazione in tedesco,Codice Ripartizione Geografica,Ripartizione geografica,Denominazione regione,...,Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Popolazione legale 2011 (09/10/2011),Codice NUTS1 2010,Codice NUTS2 2010 (3),Codice NUTS3 2010,Codice NUTS1 2006,Codice NUTS2 2006 (3),Codice NUTS3 2006,Unnamed: 25
0,8,-,39,4,39004,Brisighella,NaN,2,Nord-est,Emilia-Romagna,...,39004,B188,7.664,ITH,ITH5,ITH57,ITD,ITD5,ITD57,NaN
1,8,-,39,5,39005,Casola Valsenio,NaN,2,Nord-est,Emilia-Romagna,...,39005,B982,2.724,ITH,ITH5,ITH57,ITD,ITD5,ITD57,NaN
2,8,-,39,6,39006,Castel Bolognese,NaN,2,Nord-est,Emilia-Romagna,...,39006,C065,9.519,ITH,ITH5,ITH57,ITD,ITD5,ITD57,NaN
3,8,-,39,7,39007,Cervia,NaN,2,Nord-est,Emilia-Romagna,...,39007,C553,28.896,ITH,ITH5,ITH57,ITD,ITD5,ITD57,NaN
4,8,-,39,8,39008,Conselice,NaN,2,Nord-est,Emilia-Romagna,...,39008,C963,9.837,ITH,ITH5,ITH57,ITD,ITD5,ITD57,NaN


3. Query Facebook with the cities dataset and create mapping dict (make sure to replace the query token)

In [55]:
import requests
from urllib.parse import quote

responses_data = []
for row in df['place name'].values:
    cookies = {
        'sb': 'FVizYSX6oLSbfPsLYTEy4sK2',
        'datr': 'kC52ZUbb7Zc0B3K2xG5oqR3r',
        'c_user': '1367569598',
        'fr': '0g6qCFKlrRAXvpydD.AWW5xkjwnNXHje37dtVkdoqUAdU.Bl3zOY..AAA.0.0.Bl3zfL.AWX6M-o8y5c',
        'usida': 'eyJ2ZXIiOjEsImlkIjoiQXM5a2h2MmV5dmlvMSIsInRpbWUiOjE3MDkxMjc2Mjd9',
        'presence': 'C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1709153407400%2C%22v%22%3A1%7D',
        'dpr': '1.125',
        'oo': 'v1',
        'xs': '38%3A1CNXyg8KDP0GKA%3A2%3A1709126471%3A-1%3A10632%3A%3AAcU3gAcsh_w6MxZbHYW85ybdg-coNnSBjTJokCiN4c0',
        'wd': '1058x703',
    }
    
    headers = {
        'authority': 'www.facebook.com',
        'accept': '*/*',
        'accept-language': 'en-GB,en;q=0.9,it-IT;q=0.8,it;q=0.7,en-US;q=0.6',
        'cache-control': 'no-cache',
        'content-type': 'application/x-www-form-urlencoded',
        # 'cookie': 'sb=FVizYSX6oLSbfPsLYTEy4sK2; datr=kC52ZUbb7Zc0B3K2xG5oqR3r; c_user=1367569598; fr=0g6qCFKlrRAXvpydD.AWW5xkjwnNXHje37dtVkdoqUAdU.Bl3zOY..AAA.0.0.Bl3zfL.AWX6M-o8y5c; usida=eyJ2ZXIiOjEsImlkIjoiQXM5a2h2MmV5dmlvMSIsInRpbWUiOjE3MDkxMjc2Mjd9; presence=C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1709153407400%2C%22v%22%3A1%7D; dpr=1.125; oo=v1; xs=38%3A1CNXyg8KDP0GKA%3A2%3A1709126471%3A-1%3A10632%3A%3AAcU3gAcsh_w6MxZbHYW85ybdg-coNnSBjTJokCiN4c0; wd=1058x703',
        'dnt': '1',
        'dpr': '1.25',
        'origin': 'https://www.facebook.com',
        'pragma': 'no-cache',
        'referer': 'https://www.facebook.com/marketplace/np/109317799086039/search/?query=rhodes%20piano&radius=60&session_id=682090334&exact=false',
        'sec-ch-prefers-color-scheme': 'light',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-full-version-list': '"Chromium";v="122.0.6261.70", "Not(A:Brand";v="24.0.0.0", "Google Chrome";v="122.0.6261.70"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '""',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"10.0.0"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'viewport-width': '1058',
        'x-asbd-id': '129477',
        'x-fb-friendly-name': 'BasicMarketplaceSearchAddressDataSourceQuery',
        'x-fb-lsd': 'LHmQ5GRhEviS4_iKhsy1-9',
    }
    
    data = {
        'av': '1367569598',
        '__aaid': '0',
        '__user': '1367569598',
        '__a': '1',
        '__req': 'ci',
        '__hs': '19781.HYP:comet_pkg.2.1..2.1',
        'dpr': '1.5',
        '__ccg': 'EXCELLENT',
        '__rev': '1011712572',
        '__s': '64ctzd:xtbi57:uq1kyh',
        '__hsi': '7340757966750453463',
        '__dyn': '7AzHK4HwkEng5K8G6EjBAg2owIxu13wFG1gyUW3q2ibwNw9G2Saxa1NwJwpUe8hwaG0Z82_CxS320om78c87m221FwgolzUO0-E7m4oaEnxO0Bo7O2l2Utwwwi831wiE567Udo5qfK0zEkxe2GewGwkUtxGm2SUbElxm1Wxfxmu3W3y261eBx_wHwoE2mBwFKq2-azqwqo4i1jg2cwMwhEkxebwHwNxe6Uak0zU8oC1Hg6C13whEeE4WVU-4Edo',
        '__csr': 'htgx95R7iHHnh5HEI8iiPvkgKB9sLW9tYJEBQHN91iZSFCKKGKJf_TZXBrl__ACWGiGQiGVSF8yGAhFF8J9aAUKrGq48-l2ELGi4HyVWxu9K4k_ByU98Cii5Q-cAyrxO5obohwj88UG8G8ACDCDwWwuawhEb8XwwwUwZyo7puawDwioa8gGq3W0E86e0FUC3e1BxHwqooK0se08Ywdq06nUkxy1ow2EEhw05dDwai5FE2RQ0wU0wK0eJw4Ow3Q80hMw0Gvw7lDw1XC01y3wda3mhG05Q9m1Kw',
        '__comet_req': '15',
        'fb_dtsg': 'NAcMNhD2tl7zs1TLnobwM85DZ9ayKmG5fcsJ3qsxRedbUnMg_dmPhCw:38:1709126471',
        'jazoest': '25669',
        'lsd': 'LHmQ5GRhEviS4_iKhsy1-9',
        '__spin_r': '1011712572',
        '__spin_b': 'trunk',
        '__spin_t': '1709153402',
        '__jssesw': '1',
        'fb_api_caller_class': 'RelayModern',
        'fb_api_req_friendly_name': 'BasicMarketplaceSearchAddressDataSourceQuery',
        'variables': '{{"params":{{"query":"{}"}}}}'.format(quote(row)),
        'server_timestamps': 'true',
        'doc_id': '7026380494071631',
    }

    response = requests.post('https://www.facebook.com/api/graphql/', cookies=cookies, headers=headers, data=data)
    print(response.content)
    responses_data.append(response.json().get("data", {}))    

b'{"data":{"basic_marketplace_geo_entities_search":[{"single_line_address":"Brisighella","multi_line_address":[],"subtitle":"Citt\\u00e0","page":{"id":"103824622989212"},"location":{"latitude":44.22361,"longitude":11.77578,"reverse_geocode_detailed":{"city":"Brisighella","postal_code":"48013","country_alpha_two":"IT"}}}]},"extensions":{"is_final":true}}'
b'{"data":{"basic_marketplace_geo_entities_search":[]},"extensions":{"is_final":true}}'
b'{"data":{"basic_marketplace_geo_entities_search":[]},"extensions":{"is_final":true}}'
b'{"data":{"basic_marketplace_geo_entities_search":[{"single_line_address":"Cervia","multi_line_address":[],"subtitle":"Citt\\u00e0","page":{"id":"105762946124194"},"location":{"latitude":44.26145,"longitude":12.34739,"reverse_geocode_detailed":{"city":"Cervia","postal_code":"48015","country_alpha_two":"IT"}}},{"single_line_address":"Concerviano","multi_line_address":[],"subtitle":"Citt\\u00e0","page":{"id":"103740509665061"},"location":{"latitude":42.32277,"long

4. Build and print the cities mapping

In [68]:
city_dict = {}
for response_data in responses_data:
    for location in response_data["basic_marketplace_geo_entities_search"]:
        if location.get("single_line_address"):
            geocode_data = location["location"]["reverse_geocode_detailed"]
            postal_code = geocode_data["postal_code"]
            country_code = geocode_data["country_alpha_two"]
            city_id = location["page"]["id"]
            city_dict[(postal_code, country_code)] = city_id
print(city_dict)    

{('48013', 'IT'): '103824622989212', ('48015', 'IT'): '105762946124194', ('02020', 'IT'): '103740509665061', ('17464', 'ES'): '104804759555744', ('7950', 'BE'): '112565218760518', ('34290', 'FR'): '105958486102381', ('48017', 'IT'): '109272745759400', ('48033', 'IT'): '108326449189280', ('48018', 'IT'): '103758862995533', ('48034', 'IT'): '107403715948727', ('27004', 'ES'): '109317799086039', ('48022', 'IT'): '110105015674078', ('305500', 'RO'): '110292358990034', ('27611', 'ES'): '108204459201292', ('33420', 'ES'): '106540879380410', ('27460', 'ES'): '106038869434965', ('27320', 'ES'): '112340842112389', ('27836', 'ES'): '113166662030293', ('48121', 'IT'): '115401551805737', ('48122', 'IT'): '112979228721001', ('48125', 'IT'): '112061362152513', ('48027', 'IT'): '111752902175169'}
